<a href="https://colab.research.google.com/github/Friedrichz/crypto_tooling/blob/main/track_vc_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [209]:
!pip install messari

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [210]:
from messari.messari import Messari
import pandas as pd
import numpy as np
import urllib.request
import json
import time
# API keys
messari_api_key = "88bbfa31-1b0c-442b-b748-cc13ed2cab1a"
messari = Messari(messari_api_key)
nomics_api_key = "91714c15b8333b59ec0806fe9fcf5179410e14b2"

In [211]:
# FOR FUNDRAISING DATA USE Messari
li_dic = [] 
for i in range(1, 5, 1):
    data = messari.get_all_assets(page=i, limit=500, to_dataframe=True)
    li_dic.append(data)

# Merge raw API data into one list with records 
data = [j for i in li_dic for j in i["data"]]
len(data)

# Inspect coins
# [(n,i) for n, i in enumerate([i["name"] for i in data])]

2000

In [212]:
# USE NOMICS API TO GET MAX SUPPLY DATA: https://nomics.com/docs/#tag/Currencies-Ticker
# All tickers
ticker_list = [r["symbol"] for r in data]
ticker_list = [i.replace("$","").replace("-","").replace(".","").replace(" ","") for i in list(filter(None, ticker_list))]

def get_nomics_currency_data(ticker_list, parts=2, per_page=100):

  tl_parts = np.array_split(ticker_list, parts)
  data_list = []

  for tli in tl_parts: 
    # Make one long string
    tli_str = ','.join(tli)
    # 100 items per page resulting in n_pages
    n_ticks = len(tli)
    n_pages = int(np.ceil(n_ticks / per_page))

    # Get response from all pages
    for page in range(1, n_pages+1): 
      print(n_ticks, page)
      nomics_url = """https://api.nomics.com/v1/currencies/ticker?key={}&interval=1d&ids={}&per-page={}&page={}""".format(nomics_api_key, tli_str, per_page, page)
      response = urllib.request.urlopen(nomics_url).read()
      data_nomics = json.loads(response.decode('utf-8'))
      # Append new data
      data_list.append(data_nomics)
      time.sleep(1)

  # Unnest list of records
  data_list = [i for j in data_list for i in j]

  print(len(ticker_list))
  print(len(data_list))
  return data_list

# Fetch Nomics data
nomics_data = get_nomics_currency_data(ticker_list)
# Save Nomics currency data (For max supply)
nomics_df = pd.DataFrame.from_records(nomics_data)
# nomics_df.to_csv("nomics_currencies_data_supply.csv")

# Format for analysis
# nomics_df = nomics_df.set_index("symbol")
supply_cols = ['symbol','status','market_cap','circulating_supply','max_supply','first_trade']
nomics_df = nomics_df[supply_cols]

986 1
986 2
986 3
986 4
986 5
986 6
986 7
986 8
986 9
986 10
986 1
986 2
986 3
986 4
986 5
986 6
986 7
986 8
986 9
986 10
1972
1745


# Cleanup records 

In [213]:
def get_nested_items(list_records, attr): 
  if list_records is None:
    return []
  else: 
    return [i[attr] for i in list_records]

def clean_sales_round_title(title): 
  if title is None: pass
  else:
    # Remove digits
    title = ''.join([i for i in title if not i.isdigit()]) 
    # Lowercase & strip trailing/leading spaces
    title = title.lower().strip()
    return title

def clean_sales_rounds(list_records): 
  if list_records is None:
    pass
  else:
    [i.update(title_clean=clean_sales_round_title(i["title"])) for i in list_records]
    return

# Get list of sales_rounds labels to identify venture round / seed sale 
sales_rounds_titles = set(j for i in data for j in get_nested_items(i["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"], "title"))

# Taken manually from list of titles in data 
private_sales_rounds_titles = [
  'Fundraising ', 
  'Pre Seed Round', 
  'Pre-Sale SAFT', 
  'Private Presale', 
  'Private Sale', 
  'Private Sale1 ',
  'SAFT',
  'Seed', 
  'Seed ', 
  'Seed Funding', 
  'Seed Round', 
  'Seed Round (SAFT) ', 
  'Seed Sale', 
  'Seed Round (Crypto)', 
  'Seed Token Sale',
  'Strategic Funding Round',
  'Strategic Investments',
  'Strategic Private Sale',
  'Strategic Round',
  'Strategic Sale',
  'Token Pre-Sale',
  'Token Private Round',
  'Token Seed Round',
  'Venture Capital',
  'Venture Funding',
  'Venture Round',
]

# Clean up sales round titles & add "title_clean" to records
private_sales_rounds_titles_clean = set(clean_sales_round_title(i) for i in private_sales_rounds_titles)
print(len(private_sales_rounds_titles_clean)) # Include 24 titles that point to private token sales
_ = [clean_sales_rounds(j["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"]) for j in data]

24


# Fundraise table

In [214]:
# Extract full fundraising data for each coin
def full_fundraise_data(record):
  # Returns a list of recorded funraise events 
  symbol = record["symbol"]
  name = record["name"]

  # Get details on sales rounds
  sales_rounds = record["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"]
  if sales_rounds:
    _ = [i.update(name=name) for i in sales_rounds]
    _ = [i.update(symbol=symbol) for i in sales_rounds]

  return sales_rounds

fundraises_list = [j for i in filter(None, [full_fundraise_data(i) for i in data]) for j in i]

# Format to df
cols = ["name", "symbol", "title_clean", "type", "end_date", "native_tokens_allocated", "equivalent_price_per_token_in_usd", "amount_collected_in_usd", "asset_collected"]
fdf = pd.DataFrame.from_records(fundraises_list)[cols]
fdf = fdf.dropna(how="all", subset=["native_tokens_allocated", "equivalent_price_per_token_in_usd", "equivalent_price_per_token_in_usd"])

# Add cumulative amount raised up to & incl fundraising event
fdf["cum_amount_collected_in_usd"] = fdf.sort_values("end_date", ascending=True).groupby("symbol")["amount_collected_in_usd"].cumsum()
fdf.head()

,name,symbol,title_clean,type,end_date,native_tokens_allocated,equivalent_price_per_token_in_usd,amount_collected_in_usd,asset_collected,cum_amount_collected_in_usd
0,Ethereum,ETH,token sale,Public,2014-09-02T16:00:00Z,576000000.0,0.310,18300000.0,BTC,18300000.0
1,BNB,BNB,initial coin offering (ico),Public,2017-07-21T04:00:00Z,100000000.0,0.150,15000000.0,"ETH, BTC",15000000.0
12,Solana,SOL,seed sale,Seed,2018-04-05T00:00:00Z,79290466.0,0.040,3170000.0,USD,3170000.0
13,Solana,SOL,founding sale,Series A,2018-06-03T00:00:00Z,63151982.0,0.200,12630000.0,USD,15800000.0
14,Solana,SOL,validator sale,Series A,2019-07-09T00:00:00Z,25331653.0,0.225,5700000.0,USD,21500000.0


# Current market data

In [215]:
# Extract (current) market data --> metrics: token_sale_stats>sale_proceeds_usd, supply_distribution
def get_market_data(record): 
  # Get current supply (see docs: https://messari.io/article/messari-proprietary-methods)
  circ_supply = record["metrics"]["supply"]["circulating"] # liquid supply that excludes project, foundation or founder units which have not been yet sold
  liquid_supply = record["metrics"]["supply"]["liquid"] # supply taht is visible on-chain and which is not known to have any programmatic or contractual restrictions

  # Get ath price data
  ath_price = record["metrics"]["all_time_high"]["price"]
  ath_date = record["metrics"]["all_time_high"]["at"]

  # Get cycle low price data
  cl_price = record["metrics"]["cycle_low"]["price"]
  cl_date = record["metrics"]["cycle_low"]["at"]

  # Get current mcap
  mcap = record["metrics"]["marketcap"]["current_marketcap_usd"]
  liquid_mcap = record["metrics"]["marketcap"]["liquid_marketcap_usd"]

  # Get current price data
  current_price = record["metrics"]["market_data"]["price_usd"]

  # Get ROI data
  pct_changes_keys = ["percent_change_last_1_year", "percent_change_year_to_date", "percent_change_last_3_months", "percent_change_last_1_month", "percent_change_month_to_date", "percent_change_last_1_week",]
  pct_changes = {k:v for k,v in record["metrics"]["roi_data"].items() if k in pct_changes_keys}

  return dict(
      symbol = record["symbol"], 
      name = record["name"], 
      link = "https://messari.io/asset/{}".format(record["slug"]),
      current_price = current_price,
      circ_supply = circ_supply, 
      liquid_supply = liquid_supply, 
      ath_price = ath_price, 
      ath_date = ath_date, 
      cl_price = cl_price, 
      cl_date = cl_date, 
      mcap = mcap, 
      liquid_mcap = liquid_mcap,
      **pct_changes       
  )


# Get private fundraising data for coins (seed)

In [ ]:
# Extract initial private fundraising data -->  profile: investors>organizations, economics>launch>fundraising
# Using pandas
def get_fundraise_data(record, private_sales_rounds_titles_clean=private_sales_rounds_titles_clean):
  # Init
  investors = []
  first_private_sale_date = np.nan
  last_private_sale_date = np.nan
  num_tokens_private_sale = np.nan
  tot_usd_raised_private_sale = np.nan
  min_price_private_sale = np.nan
  max_price_private_sale = np.nan
  wavg_price_private_sale = np.nan 
  init_supply = np.nan
  fdv_private_sale = np.nan
  asset_collected = np.nan

  # Get total token supply
  init_supply = record["profile"]["economics"]["launch"]["initial_distribution"]["initial_supply"]

  # Get list of private investors
  coin_investors = record["profile"]["investors"]["organizations"] 
  if coin_investors != None:
    investors = [i["name"] for i in coin_investors]

  # Get details on sales rounds
  sales_rounds = record["profile"]["economics"]["launch"]["fundraising"]["sales_rounds"]
  if sales_rounds:
    df = pd.DataFrame.from_records(sales_rounds)

    # Filter records of private token sales
    df1 = df[df.title_clean.isin(private_sales_rounds_titles_clean)]
    # Get earliest & last date of private sales
    first_private_sale_date = df1.end_date.astype(str).min()
    last_private_sale_date = df1.end_date.astype(str).max()

    # Calc total number of tokens sold during private sales
    num_tokens_private_sale = df1.native_tokens_allocated.sum()
    # Calc total USD amount raised in private sales
    tot_usd_raised_private_sale = df1.amount_collected_in_usd.sum()

    # Get min/max private sale token price 
    min_price_private_sale = df1.equivalent_price_per_token_in_usd.min()
    max_price_private_sale = df1.equivalent_price_per_token_in_usd.max()

    # Calc weighted avg token price of private sales
    if num_tokens_private_sale:
      wavg_price_private_sale = sum(df1.equivalent_price_per_token_in_usd * df1.native_tokens_allocated) / num_tokens_private_sale

    # Calc FDV based on last private sale
    if init_supply:
      fdv_private_sale = max_price_private_sale * init_supply

    # Safety check: currency collected in private fundraise
    asset_collected = df1.asset_collected.unique().tolist()

  return dict(
        investors = investors, 
        first_private_sale_date = first_private_sale_date, 
        last_private_sale_date = last_private_sale_date, 
        num_tokens_private_sale = num_tokens_private_sale, 
        tot_usd_raised_private_sale = tot_usd_raised_private_sale, 
        min_price_private_sale = min_price_private_sale, 
        max_price_private_sale = max_price_private_sale, 
        wavg_price_private_sale = wavg_price_private_sale, 
        init_supply = init_supply, 
        fdv_private_sale = fdv_private_sale,
        asset_collected = asset_collected,
    )


# Generate tables

In [216]:
fdf

,name,symbol,title_clean,type,end_date,native_tokens_allocated,equivalent_price_per_token_in_usd,amount_collected_in_usd,asset_collected,cum_amount_collected_in_usd
0,Ethereum,ETH,token sale,Public,2014-09-02T16:00:00Z,5.760000e+08,0.31000,18300000.0,BTC,18300000.0
1,BNB,BNB,initial coin offering (ico),Public,2017-07-21T04:00:00Z,1.000000e+08,0.15000,15000000.0,"ETH, BTC",15000000.0
12,Solana,SOL,seed sale,Seed,2018-04-05T00:00:00Z,7.929047e+07,0.04000,3170000.0,USD,3170000.0
13,Solana,SOL,founding sale,Series A,2018-06-03T00:00:00Z,6.315198e+07,0.20000,12630000.0,USD,15800000.0
14,Solana,SOL,validator sale,Series A,2019-07-09T00:00:00Z,2.533165e+07,0.22500,5700000.0,USD,21500000.0
...,...,...,...,...,...,...,...,...,...,...
634,BENQI,QI,public sale a,Public Sale,2021-04-28T00:00:00Z,4.392000e+08,0.00750,3294000.0,USD,4374000.0
635,BENQI,QI,public sale b,Public Sale,2021-04-28T00:00:00Z,6.480000e+07,0.00900,583200.0,USD,4957200.0
636,BENQI,QI,private sale,Private Sale,2021-04-28T00:00:00Z,9.360000e+08,0.00550,5148000.0,USD,10105200.0
638,Permission,ASK,saft (saftlaunch),Public,2018-03-02T05:00:00Z,1.796918e+09,0.00072,1287431.0,ETH,1287431.0


In [228]:
# Make df with current market data (price, mcap, circ supply, first trade, max supply, status) 
mkt_data = [get_market_data(record) for record in data]
df_mkt_data = pd.DataFrame.from_records(mkt_data)
# Clean symbols, drop dups
df_mkt_data.symbol =  df_mkt_data.symbol.str.replace("$","").replace("-","").replace(".","").replace(" ","").str.upper().astype(str)
nomics_df = nomics_df.drop_duplicates("symbol")
print(df_mkt_data.shape)
print(nomics_df.shape)

# Join dfs: Add first trade, max supply, status from nomics
df_mkt_data1 = df_mkt_data.merge(nomics_df, how="left", left_on="symbol", right_on="symbol")
print(df_mkt_data1.shape)

# Current market data set 
numeric_cols = ["market_cap", "circulating_supply", "max_supply"]
df_mkt_data1[numeric_cols] = df_mkt_data1[numeric_cols].astype(float)
df_mkt_data1["fdv_current"] = df_mkt_data1.max_supply * df_mkt_data1.current_price

(2000, 18)
(1708, 6)
(2000, 23)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """


In [231]:
# Valuations at fundraising event vs current valuation (mcap, fdv)
print(fdf.shape)
print(df_mkt_data1.shape)

fvals = fdf.merge(df_mkt_data1, how="left", left_on="symbol", right_on="symbol")
print(fvals.shape)

# Calc return / valuation delta since fundraise event 
fvals["fdv_at_raise"] = fvals.max_supply * fvals.equivalent_price_per_token_in_usd
fvals["val_current_delta"] = fvals.fdv_current / fvals.fdv_at_raise -1

fvals.head()

(362, 10)
(2000, 24)
(384, 33)


,name_x,symbol,title_clean,type,end_date,native_tokens_allocated,equivalent_price_per_token_in_usd,amount_collected_in_usd,asset_collected,cum_amount_collected_in_usd,...,percent_change_month_to_date,percent_change_year_to_date,status,market_cap,circulating_supply,max_supply,first_trade,fdv_current,fdv_at_raise,val_current_delta
0,Ethereum,ETH,token sale,Public,2014-09-02T16:00:00Z,576000000.0,0.310,18300000.0,BTC,18300000.0,...,1.371784,-56.160657,active,2.019448e+11,121814924.0,NaN,2015-08-07T00:00:00Z,NaN,NaN,NaN
1,BNB,BNB,initial coin offering (ico),Public,2017-07-21T04:00:00Z,100000000.0,0.150,15000000.0,"ETH, BTC",15000000.0,...,4.919756,-43.725770,active,4.938605e+10,163276975.0,163276975.0,2017-07-14T00:00:00Z,4.951128e+10,2.449155e+07,2020.566207
2,Solana,SOL,seed sale,Seed,2018-04-05T00:00:00Z,79290466.0,0.040,3170000.0,USD,3170000.0,...,-4.189798,-77.739109,active,1.392644e+10,346652647.0,508180964.0,2020-04-10T00:00:00Z,2.041025e+10,2.032724e+07,1003.083917
3,Solana,SOL,founding sale,Series A,2018-06-03T00:00:00Z,63151982.0,0.200,12630000.0,USD,15800000.0,...,-4.189798,-77.739109,active,1.392644e+10,346652647.0,508180964.0,2020-04-10T00:00:00Z,2.041025e+10,1.016362e+08,199.816783
4,Solana,SOL,validator sale,Series A,2019-07-09T00:00:00Z,25331653.0,0.225,5700000.0,USD,21500000.0,...,-4.189798,-77.739109,active,1.392644e+10,346652647.0,508180964.0,2020-04-10T00:00:00Z,2.041025e+10,1.143407e+08,177.503807


In [236]:
fvals.columns

Index(['name_x', 'symbol', 'title_clean', 'type', 'end_date',
       'native_tokens_allocated', 'equivalent_price_per_token_in_usd',
       'amount_collected_in_usd', 'asset_collected',
       'cum_amount_collected_in_usd', 'name_y', 'link', 'current_price',
       'circ_supply', 'liquid_supply', 'ath_price', 'ath_date', 'cl_price',
       'cl_date', 'mcap', 'liquid_mcap', 'percent_change_last_1_week',
       'percent_change_last_1_month', 'percent_change_last_3_months',
       'percent_change_last_1_year', 'percent_change_month_to_date',
       'percent_change_year_to_date', 'status', 'market_cap',
       'circulating_supply', 'max_supply', 'first_trade', 'fdv_current',
       'fdv_at_raise', 'val_current_delta'],
      dtype='object')

In [237]:
fvals[['symbol', 'title_clean', 'type', 'end_date', 'equivalent_price_per_token_in_usd', 'cum_amount_collected_in_usd','current_price',
       'circulating_supply', 'max_supply', 'first_trade', 'fdv_current', 'fdv_at_raise', 'val_current_delta']]

,symbol,title_clean,type,end_date,equivalent_price_per_token_in_usd,cum_amount_collected_in_usd,current_price,circulating_supply,max_supply,first_trade,fdv_current,fdv_at_raise,val_current_delta
0,ETH,token sale,Public,2014-09-02T16:00:00Z,0.31000,18300000.0,1658.747435,121814924.0,NaN,2015-08-07T00:00:00Z,NaN,NaN,NaN
1,BNB,initial coin offering (ico),Public,2017-07-21T04:00:00Z,0.15000,15000000.0,303.234931,163276975.0,1.632770e+08,2017-07-14T00:00:00Z,4.951128e+10,2.449155e+07,2020.566207
2,SOL,seed sale,Seed,2018-04-05T00:00:00Z,0.04000,3170000.0,40.163357,346652647.0,5.081810e+08,2020-04-10T00:00:00Z,2.041025e+10,2.032724e+07,1003.083917
3,SOL,founding sale,Series A,2018-06-03T00:00:00Z,0.20000,15800000.0,40.163357,346652647.0,5.081810e+08,2020-04-10T00:00:00Z,2.041025e+10,1.016362e+08,199.816783
4,SOL,validator sale,Series A,2019-07-09T00:00:00Z,0.22500,21500000.0,40.163357,346652647.0,5.081810e+08,2020-04-10T00:00:00Z,2.041025e+10,1.143407e+08,177.503807
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,QI,public sale a,Public Sale,2021-04-28T00:00:00Z,0.00750,4374000.0,0.017025,NaN,NaN,2021-02-17T00:00:00Z,NaN,NaN,NaN
380,QI,public sale b,Public Sale,2021-04-28T00:00:00Z,0.00900,4957200.0,0.017025,NaN,NaN,2021-02-17T00:00:00Z,NaN,NaN,NaN
381,QI,private sale,Private Sale,2021-04-28T00:00:00Z,0.00550,10105200.0,0.017025,NaN,NaN,2021-02-17T00:00:00Z,NaN,NaN,NaN
382,ASK,saft (saftlaunch),Public,2018-03-02T05:00:00Z,0.00072,1287431.0,NaN,NaN,1.000000e+12,2019-02-22T00:00:00Z,NaN,7.200000e+08,NaN


In [ ]:
# Get final data for all records
final_records = []

for record in data: 
  final_records.append(
      dict(
        symbol = record["symbol"], 
        name = record["name"], 
        link = "https://messari.io/asset/{}".format(record["slug"]),
        ** get_market_data(record),
        ** get_fundraise_data(record), 
      )
  )

In [ ]:
# Create pd df
df = pd.DataFrame.from_records(final_records)
print(df.shape)

# Add ROI column since private sale
df["pct_change_private_sale"] = (df.current_price / df.wavg_price_private_sale - 1) * 100
df["pct_change_valuation"] = (df.mcap / df.fdv_private_sale - 1) * 100

df.head()

(2000, 29)


,symbol,name,link,current_price,circ_supply,liquid_supply,ath_price,ath_date,cl_price,cl_date,...,num_tokens_private_sale,tot_usd_raised_private_sale,min_price_private_sale,max_price_private_sale,wavg_price_private_sale,init_supply,fdv_private_sale,asset_collected,pct_change_private_sale,pct_change_valuation
0,BTC,Bitcoin,https://messari.io/asset/bitcoin,23208.090131,1.910791e+07,1.912012e+07,68721.934821,2021-11-10T14:00:00Z,17664.958903,2022-06-18T20:15:00Z,...,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN
1,ETH,Ethereum,https://messari.io/asset/ethereum,1679.456694,1.217644e+08,1.165134e+08,4847.573312,2021-11-10T15:30:00Z,897.060060,2022-06-18T20:30:00Z,...,0.0,0.0,NaN,NaN,NaN,7.200368e+07,NaN,[],NaN,NaN
2,USDT,Tether,https://messari.io/asset/tether,0.999446,6.605698e+10,NaN,1.261429,2016-02-12T08:30:00Z,0.000021,2022-07-02T12:30:01Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USDC,USD Coin,https://messari.io/asset/usd-coin,1.000000,5.445763e+10,NaN,1.127539,2018-11-18T09:15:00Z,0.872572,2019-10-03T18:30:00Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BNB,BNB,https://messari.io/asset/binance-coin,284.816286,1.613373e+08,1.083456e+08,689.922433,2021-05-10T06:15:00Z,185.648220,2022-06-18T20:15:00Z,...,0.0,0.0,NaN,NaN,NaN,2.000000e+08,NaN,[],NaN,NaN


In [ ]:
df[df.init_supply > 0]

(272, 31)